In [ ]:
// KNN

// link knn - https://www.kaggle.com/datasets/uciml/faulty-steel-plates?resource=download
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))

print("Dataset Loaded Successfully")
print(df.head())


label_cols = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
fault_names = label_cols


def get_fault(row):
    for i, col in enumerate(label_cols):
        if row[col] == 1:
            return fault_names[i]
    return "Unknown"

df['Fault_Type'] = df[label_cols].apply(get_fault, axis=1)


X = df.drop(label_cols + ['Fault_Type'], axis=1)
y = df['Fault_Type']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)


print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=fault_names,
    yticklabels=fault_names
)
plt.title("Confusion Matrix - KNN")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()


k_vals = range(1, 15)
accuracies = []

for k in k_vals:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    accuracies.append(model.score(X_test, y_test))

plt.figure(figsize=(8,6))
plt.plot(k_vals, accuracies, marker='o')
plt.title("Accuracy vs K Value")
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.grid(True)
plt.tight_layout()
plt.show()

print("\nBest K =", k_vals[np.argmax(accuracies)],
      "with accuracy =", max(accuracies))
